This code was copied from a Jupyter notebook from Rich Signell's youtube video: https://www.youtube.com/watch?v=2TkZa0s58qs

In [ ]:
import xarray as xr
import s3fs
from geoviews import opts
import hvplot.pandas
import hvplot.xarray
import geoviews as gv

In [ ]:
fs = s3fs.S3FileSystem(anon=True)
s3map = s3fs.S3Map('esip-pangeo-uswest2/pangeo/NWM/2010', s3=fx)
ds = xr.open_zarr(s3map)

In [ ]:
fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
fileObj = fs.open('esip-pangeo-uswest2/pangeo/NWM/nwm-v1.2-channel_spatial_index.nc')

In [ ]:
ds_lonlat=xr.open_dataset(fileObj)

In [ ]:
ds.coords['latitude'] = ds_lonlat['latitude']
ds.coords['longitude'] = ds_lonlat['longitude']

In [ ]:
ds.time[-1]

In [ ]:
ds.streamflow

In [ ]:
ds.streamflow[:, 0].hvplot()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from dask.distributed import Client, progress, LocalCluster
from dask_kubernetes import KubeCluster

In [ ]:
cluster = KubeCluster()
cluster.scale(25)
cluster

In [ ]:
client = Client(cluster)

In [ ]:
var = 'streamflow'

In [ ]:
ds[var].nbytes/1e9

In [ ]:
var_mean = ds[var].mean(dim='time').persist()
progress(var_mean)

In [ ]:
df = var_mean.to_pandas().to_frame()

In [ ]:
df = df.assign(latitude=df['latitude'].values)
df = df.assign(longitude=df['longitude'].values)
df.rename(columns={0:"transport"}, inplace=True)

In [ ]:
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize, spread

import cartopy.crs as ccrs

In [ ]:
p = df.hvplot.points('longitude', 'latitude', crs=ccrs.PlateCarree(),
                     c='transport', colorbar=True, size=14)

In [ ]:
g = rasterize(p, aggregator='mean', x_sampling=0.02, y_sampling=0.02,
              width=500).opts(tools=['hover'])

In [ ]:
g * gv.tile_sources.OSM